In [1]:
import torchvision.datasets as datasets
import torch
import matplotlib.pyplot as plt
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import warnings
warnings.filterwarnings('ignore')

2024-05-15 17:55:20.269189: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-15 17:55:20.914786: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
writer = SummaryWriter('alex_logs')

In [3]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((227, 227)),  
    transforms.ToTensor(),           
    transforms.Normalize(            
        mean=[0.485, 0.456, 0.406],  
        std=[0.229, 0.224, 0.225]   
    )
])

train_data = datasets.Imagenette(root='data', split='train', download=False, transform=transform)
test_data = datasets.Imagenette(root='data', split='val', download=False, transform=transform)

In [4]:
len(train_data.classes)

10

In [5]:
class AlexNet(nn.Module):

    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 96, (11, 11), stride=4)
        self.conv2 = nn.Conv2d(96, 256, (5, 5), stride=1, padding=2)
        self.conv3 = nn.Conv2d(256, 384, (3, 3), stride=1, padding=1)
        self.conv4 = nn.Conv2d(384, 384, (3, 3), stride=1, padding=1)
        self.conv5 = nn.Conv2d(384, 256, (3, 3), stride=1, padding=1)

        self.fc1 = nn.Linear(6*6*256, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 16)

    def forward(self, x):
        print(x.shape)
        out = F.relu(self.conv1(x))
        print(out.shape)
        out = F.max_pool2d(out, (3, 3), stride=2)
        print(out.shape)
        out = F.relu(self.conv2(out))
        print(out.shape)
        out = F.max_pool2d(out, (3, 3), stride=2)
        print(out.shape)
        out = F.relu(self.conv3(out))
        print(out.shape)
        out = F.relu(self.conv4(out))
        print(out.shape)
        out = F.relu(self.conv5(out))
        print(out.shape)
        out = F.max_pool2d(out, (3, 3), stride=2)
        print(out.shape)
        out = out.view(out.size(0), -1)
        print(out.shape)
        out = F.dropout2d(out, 0.5)
        print(out.shape)


        
        out = F.relu(self.fc1(out))
        out = F.dropout2d(out, 0.5)
        out = F.relu(self.fc2(out))
        out = F.log_softmax(self.fc3(out))
        return out

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
model = AlexNet().to(device)

In [8]:
print(model)

AlexNet(
  (conv1): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=9216, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=4096, bias=True)
  (fc3): Linear(in_features=4096, out_features=16, bias=True)
)


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [10]:
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(test_data, batch_size=16, shuffle=True)


In [11]:
num_epochs = 10
for epoch in range(num_epochs):
    # Training
    model.train()
    running_loss = 0.0
    total_correct = 0
    total_samples = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        total_correct += (torch.max(outputs, 1)[1] == labels).sum().item()
        total_samples += labels.size(0)
        if i % 100 == 99:    # Print every 100 mini-batches
            print('[%d, %5d] train loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            # Log the running loss to TensorBoard
            writer.add_scalar('training loss', running_loss / 100, epoch * len(train_loader) + i)
            running_loss = 0.0

    # Calculate training accuracy
    train_accuracy = total_correct / total_samples

    # Validation
    model.eval()
    val_loss = 0.0
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            total_correct += (torch.max(outputs, 1)[1] == labels).sum().item()
            total_samples += labels.size(0)

    # Calculate validation loss and accuracy
    val_loss /= len(val_loader)
    val_accuracy = total_correct / total_samples

    print('[%d] val loss: %.3f, train accuracy: %.3f, val accuracy: %.3f' % (epoch + 1, val_loss, train_accuracy, val_accuracy))
    # Log validation loss and accuracy to TensorBoard
    writer.add_scalar('validation loss', val_loss, epoch)
    writer.add_scalar('training accuracy', train_accuracy, epoch)
    writer.add_scalar('validation accuracy', val_accuracy, epoch)

torch.Size([16, 3, 227, 227])
torch.Size([16, 96, 55, 55])
torch.Size([16, 96, 27, 27])
torch.Size([16, 256, 27, 27])
torch.Size([16, 256, 13, 13])
torch.Size([16, 384, 13, 13])
torch.Size([16, 384, 13, 13])
torch.Size([16, 256, 13, 13])
torch.Size([16, 256, 6, 6])
torch.Size([16, 9216])
torch.Size([16, 9216])
torch.Size([16, 3, 227, 227])
torch.Size([16, 96, 55, 55])
torch.Size([16, 96, 27, 27])
torch.Size([16, 256, 27, 27])
torch.Size([16, 256, 13, 13])
torch.Size([16, 384, 13, 13])
torch.Size([16, 384, 13, 13])
torch.Size([16, 256, 13, 13])
torch.Size([16, 256, 6, 6])
torch.Size([16, 9216])
torch.Size([16, 9216])


OutOfMemoryError: CUDA out of memory. Tried to allocate 144.00 MiB. GPU 